# Data cleaning for scraped cars

### Read dataset

In [3]:
import pandas as pd

In [4]:
data = pd.read_json('./car_scraper/cars.json')

In [5]:
data.head()

,brand_and_model,price,province,matriculation,fuel,kilometres,gear
0,AUDI A5 Coupé 3.0TDI quattro S-Tronic,15.450,Granada,2010,Diésel,303.000 km,Automático
1,BMW X3 xDrive 20iA,57.500,Almería,2021,Gasolina,10 km,Automático
2,OPEL Mokka X 1.4T Innovation 4x2 Aut.,13.499,Castellón,2019,Gasolina,98.456 km,Automático
3,BMW Serie 3 318dA Essential Edition,13.990,Cantabria,2014,Diésel,192.000 km,Automático
4,PEUGEOT 308 SW 1.6 BlueHDi MBC Active 120,10.200,Cádiz,2016,Diésel,134.000 km,Manual


In [6]:
data.shape

(9984, 7)

In [7]:
data.dtypes

brand_and_model    object
price              object
province           object
matriculation       int64
fuel               object
kilometres         object
gear               object
dtype: object

### Clean price

In [8]:
data[data['price']=='Consultar ']

,brand_and_model,price,province,matriculation,fuel,kilometres,gear
7376,JAGUAR XJ XJ6 2.7D V6 Executive Aut.,Consultar,Vizcaya,2006,Diésel,198.000 km,Automático


In [9]:
data = data[data['price']!='Consultar ']

In [10]:
data['price'] = [int(price[:-2].replace('.','')) for price in data['price']]

### Clean kilometres

In [11]:
data[data['kilometres']=='<li>Cambio: <span>Automático']

,brand_and_model,price,province,matriculation,fuel,kilometres,gear
1679,JAGUAR XE 2.0 Diesel Mid 180cv Auto.Prestige,24000,Vizcaya,2016,Diésel,<li>Cambio: <span>Automático,<li>Potencia: <span>180 cv
9429,LEXUS UX 250h Executive Navigation 2WD,30900,Asturias,2019,<li>Kilómetros: <span>17.000 km,<li>Cambio: <span>Automático,<li>Potencia: <span>184 cv


In [12]:
data = data[data['kilometres']!='<li>Cambio: <span>Automático']

In [13]:
data['kilometres'] = [int(kms[:-3].replace('.','')) for kms in data['kilometres']]

###  Explore fuel and gear

In [49]:
data['fuel'].value_counts()

Diésel                4741
Gasolina              4106
Híbrido                709
Eléctrico              284
Híbrido Enchufable     101
Gas                     39
Name: fuel, dtype: int64

In [48]:
data['fuel'] = [i[1:] for i in data['fuel']]

In [15]:
data['gear'].value_counts()

Manual                        5489
Automático                    4491
<li>Potencia: <span>190 cv       1
Name: gear, dtype: int64

In [16]:
data = data[data['gear']!='<li>Potencia: <span>190 cv']

### Explore matriculation and province

In [17]:
data['matriculation'].value_counts()

2021    1717
2017    1526
2018    1451
2019    1292
2020    1163
2016     845
2015     430
2014     264
2013     212
2012     164
2011     139
2010     126
2007      99
2009      94
2008      92
2005      73
2006      69
2022      48
2003      42
2004      39
2002      23
2000      20
2001      16
1999      10
1997       6
1994       5
1995       4
1998       4
1992       3
1996       2
1993       2
Name: matriculation, dtype: int64

In [18]:
data['province'].value_counts()

Madrid                 1819
Barcelona              1440
Vizcaya                 835
Valencia                816
Sevilla                 475
Alicante                432
Murcia                  396
Málaga                  384
Pontevedra              330
Zaragoza                204
Castellón               197
Toledo                  184
Girona                  157
Cádiz                   148
Granada                 137
La Coruña               135
Valladolid              128
Córdoba                 113
Asturias                113
Badajoz                 109
Jaén                    108
Guipúzcoa               104
Cantabria                97
Salamanca                96
Tarragona                79
Cáceres                  76
Lugo                     69
Burgos                   63
Navarra                  62
Almería                  59
Guadalajara              55
Ciudad Real              55
Huelva                   52
Huesca                   52
Islas Baleares           45
Las Palmas          

### Separate brand and model

In [19]:
data['brand_and_model'][0][2:-2].split(sep=' ', maxsplit=1)

['AUDI', 'A5 Coupé 3.0TDI quattro S-Tronic']

In [20]:
data['brand_and_model'] = [brand_and_model[2:-2] for brand_and_model in data['brand_and_model']]

In [21]:
data[['brand','model']] = data['brand_and_model'].str.split(' ', n=1, expand=True)

In [22]:
data = data.drop(['brand_and_model'], axis=1)
data = data[['brand', 'model', 'price', 'province', 'matriculation', 'fuel', 'kilometres', 'gear']]

### Explore brand

In [23]:
data['brand'].value_counts()

BMW              937
AUDI             848
MERCEDES-BENZ    810
VOLKSWAGEN       723
PEUGEOT          640
TOYOTA           595
FORD             586
OPEL             470
CITROEN          468
SEAT             467
RENAULT          451
KIA              280
NISSAN           273
HYUNDAI          253
FIAT             245
MINI             210
VOLVO            194
LAND-ROVER       193
SKODA            144
PORSCHE          144
LEXUS            137
MAZDA            118
JAGUAR           110
SMART             84
JEEP              73
HONDA             70
DACIA             64
ALFA              50
DS                45
MITSUBISHI        44
SUZUKI            37
SSANGYONG         33
MASERATI          27
ABARTH            20
INFINITI          20
CUPRA             18
CHEVROLET         17
FERRARI           14
TESLA             13
CHRYSLER           9
BENTLEY            8
DR                 6
SAAB               5
ASTON              5
DODGE              4
SUBARU             4
LAMBORGHINI        3
LANCIA       

### Add environmental label

In [30]:
def determine_label(matriculation, fuel):
    if fuel == 'Eléctrico' or fuel=='Híbrido enchufable':
        return '0'
    elif fuel == 'Gas' or fuel=='Híbrido':
        return 'ECO'
    elif (matriculation >= 2006 and fuel=='Gasolina') or (matriculation >= 2015 and fuel=='Diésel'):
        return 'C'
    elif (matriculation >= 2001 and fuel=='Gasolina') or (matriculation >= 2006 and fuel=='Diésel'):
        return 'B'
    else:
        return 'A'   

In [53]:
data['enviromental_label'] = [determine_label(data['matriculation'][i], data['fuel'][i]) for i in data.index]

In [54]:
data['enviromental_label'].value_counts()

C      7764
B       945
ECO     748
0       284
A       239
Name: enviromental_label, dtype: int64

In [51]:
data.head()

,brand,model,price,province,matriculation,fuel,kilometres,gear,enviromental_label
0,AUDI,A5 Coupé 3.0TDI quattro S-Tronic,15450,Granada,2010,Diésel,303000,Automático,B
1,BMW,X3 xDrive 20iA,57500,Almería,2021,Gasolina,10,Automático,C
2,OPEL,Mokka X 1.4T Innovation 4x2 Aut.,13499,Castellón,2019,Gasolina,98456,Automático,C
3,BMW,Serie 3 318dA Essential Edition,13990,Cantabria,2014,Diésel,192000,Automático,B
4,PEUGEOT,308 SW 1.6 BlueHDi MBC Active 120,10200,Cádiz,2016,Diésel,134000,Manual,C


In [55]:
data.to_csv('clean_cars_data.csv')